In [101]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:

def scrape_stilltasty(index):
    # URL of the page to scrape
    url = f'https://www.stilltasty.com/Fooditems/index/{index}'

    # Send a GET request to the URL
    response = requests.get(url)

    # Initialize a dictionary to store values
    shelf_life = {}

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find first storage location
        storage_location_elements = soup.find_all('div', class_='food-storage-left')

        for e in storage_location_elements:
            storage_location = e.text.strip()
            # Find associated shelf life information
            shelf_life_info_element = e.find_next_sibling('div')
            shelf_life_info = shelf_life_info_element.text.strip()
            shelf_life[storage_location] = shelf_life_info

        return shelf_life
    else:
        # Print an error message if the request was not successful
        print(f"Error: Failed to retrieve data from {url}. Status code: {response.status_code}")

In [98]:
def get_index_numbers(url,target_string,get_names=False):
    index_numbers = []
    object_names = []

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all links with href attribute containing target_string
        links = soup.find_all('a', href=lambda href: href and target_string in href)
        links = [x for i,x in enumerate(links) if i%2 != 0]
        # Extract index numbers from href attributes
        for link in links:
            href = link['href']
            index_number = href.split('/')[-1]
            index_numbers.append(index_number)
            if get_names:
                object_name = link.text.strip()
                object_names.append(object_name)
            
        return index_numbers, object_names 
    
    else:
        # Print an error message if the request was not successful
        print(f"Error: Failed to retrieve data from {url}. Status code: {response.status_code}")

In [93]:
url = 'https://www.stilltasty.com/Fooditems/index'
target_string = "/searchitems/index/"
index_numbers = []
object_names = []

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all links with href attribute containing target_string
    links = soup.find_all('a', href=lambda href: href and target_string in href)
    links = [x for i,x in enumerate(links) if i%2 == 1]
    # Extract index numbers from href attributes
    for link in links:
        href = link['href']
        index_number = href.split('/')[-1]
        index_numbers.append(index_number)
        object_name = link.text.strip()
        object_names.append(object_name)

    print(index_numbers)  
    print(object_names) 

else:
    # Print an error message if the request was not successful
    ValueError(f"Error: Failed to retrieve data from {url}. Status code: {response.status_code}")

['26', '25', '9', '27', '7', '28', '6', '31', '30', '5']
['Fruits', 'Vegetables', 'Dairy & Eggs', 'Meat & Poultry', 'Fish & Shellfish', 'Nuts, Grains & Pasta', 'Condiments & Oils', 'Snacks & Baked Goods', 'Herbs & Spices', 'Beverages']


In [92]:
links

[<a href="https://www.stilltasty.com/searchitems/index/26">Fruits</a>,
 <a href="https://www.stilltasty.com/searchitems/index/25">Vegetables</a>,
 <a href="https://www.stilltasty.com/searchitems/index/9">Dairy &amp; Eggs</a>,
 <a href="https://www.stilltasty.com/searchitems/index/27">Meat &amp; Poultry</a>,
 <a href="https://www.stilltasty.com/searchitems/index/7">Fish &amp; Shellfish</a>,
 <a href="https://www.stilltasty.com/searchitems/index/28">Nuts, Grains &amp; Pasta</a>,
 <a href="https://www.stilltasty.com/searchitems/index/6">Condiments &amp; Oils</a>,
 <a href="https://www.stilltasty.com/searchitems/index/31">Snacks &amp; Baked Goods</a>,
 <a href="https://www.stilltasty.com/searchitems/index/30">Herbs &amp; Spices</a>,
 <a href="https://www.stilltasty.com/searchitems/index/5">Beverages</a>]

In [99]:
# URL of the StillTasty index page
url = 'https://www.stilltasty.com/Fooditems/index'
initial_index_numbers, categories = get_index_numbers(url,"/searchitems/index/",True)
print(initial_index_numbers)
print(categories)

['26', '25', '9', '27', '7', '28', '6', '31', '30', '5']
['Fruits', 'Vegetables', 'Dairy & Eggs', 'Meat & Poultry', 'Fish & Shellfish', 'Nuts, Grains & Pasta', 'Condiments & Oils', 'Snacks & Baked Goods', 'Herbs & Spices', 'Beverages']


In [102]:
all_index_numbers = []
all_item_names = []
for i in initial_index_numbers:
    print(f"Now scraping index {i}")
    url = f'https://www.stilltasty.com/searchitems/index/{i}'
    index_numbers, item_names = get_index_numbers(url,"/Fooditems/index/")
    if index_numbers:
        all_index_numbers = all_index_numbers + index_numbers
        all_item_names = all_item_names + item_names
    print(f'Completed index {i}')
    sleep(3)
    other_pages, page_names = get_index_numbers(url,"/searchitems/index/")
    print(other_pages)
    if other_pages:
        for j in other_pages: 
            print(f"Now scraping index {j}")
            url = f'https://www.stilltasty.com/searchitems/index/{i}'
            index_numbers, item_names = get_index_numbers(url,"/Fooditems/index/")
            if index_numbers:
                all_index_numbers = all_index_numbers + index_numbers
                all_item_names = all_item_names + item_names
            print(f'Completed index {j}')
            sleep(3)
print(all_index_numbers)
print(all_item_names)

Now scraping index 26
Completed index 26
['26?page=3', '26?page=5', '26?page=7', '26?page=9']
Now scraping index 26?page=3
Completed index 26?page=3
Now scraping index 26?page=5
Completed index 26?page=5
Now scraping index 26?page=7
Completed index 26?page=7
Now scraping index 26?page=9
Completed index 26?page=9
Now scraping index 25
Completed index 25
['25?page=3', '25?page=5', '25?page=7', '25?page=9']
Now scraping index 25?page=3
Completed index 25?page=3
Now scraping index 25?page=5
Completed index 25?page=5
Now scraping index 25?page=7
Completed index 25?page=7
Now scraping index 25?page=9
Completed index 25?page=9


In [71]:
scrape_stilltasty('17634')

{'Refrigerator': '3-4 days', 'Freezer': '10-12 months'}